# Homework 2 - Bike Sharing
# Adam Resnick
# CSCI 7000 - Applied Machine Learning



### Bike Sharing
### Dataset URL: https://www.kaggle.com/marklvl/bike-sharing-dataset
### Code adapted from: https://github.com/haydarozler/bike-sharing-prediction-with-RNN/blob/master/BikeSharing-V3.ipynb

## Loading Libraries

In [62]:
# Import data processing tools
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib ipympl

import numpy as np
from numpy import array
from numpy import hstack
import keras as k

# Import ML tools
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.recurrent import SimpleRNN, LSTM
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.optimizers import Adam
from keras import regularizers
from keras.utils import np_utils
import matplotlib.pyplot as plt


# Import tools needed for visualization
from IPython.display import Image, display

np.random.seed(101) # for reproducibility”
#################################################

from keras.datasets import cifar10

### Importing and Plotting Data

In [5]:
#import kaggle
#kaggle.api.authenticate()
#kaggle.api.dataset_download_files('day', path='the_path_you_want_to_download_the_files_to', unzip=True)

dataset = pd.read_csv('C:\\Users\\Adam\\CSCI 7000 - AML\\day.csv')

In [6]:
dataset

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985
1,2,2011-01-02,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131,670,801
2,3,2011-01-03,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349
3,4,2011-01-04,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,108,1454,1562
4,5,2011-01-05,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,82,1518,1600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,727,2012-12-27,1,1,12,0,4,1,2,0.254167,0.226642,0.652917,0.350133,247,1867,2114
727,728,2012-12-28,1,1,12,0,5,1,2,0.253333,0.255046,0.590000,0.155471,644,2451,3095
728,729,2012-12-29,1,1,12,0,6,0,2,0.253333,0.242400,0.752917,0.124383,159,1182,1341
729,730,2012-12-30,1,1,12,0,0,0,1,0.255833,0.231700,0.483333,0.350754,364,1432,1796


In [7]:
plt.figure(figsize=(10,5))
plt.plot(dataset['cnt'], color='gray')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Investigating outliers, replacing with value from following week on same day

In [18]:
dataset.nsmallest(10, 'cnt')

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
667,668,2012-10-29,4,1,10,0,1,1,3,0.440000,0.439400,0.880000,0.358200,2,20,22
26,27,2011-01-27,1,0,1,0,4,1,1,0.195000,0.219700,0.687500,0.113837,15,416,431
725,726,2012-12-26,1,1,12,0,3,1,3,0.243333,0.220333,0.823333,0.316546,9,432,441
25,26,2011-01-26,1,0,1,0,3,1,3,0.217500,0.203600,0.862500,0.293850,34,472,506
64,65,2011-03-06,1,0,3,0,0,0,2,0.376522,0.366252,0.948261,0.343287,114,491,605
68,69,2011-03-10,1,0,3,0,4,1,3,0.389091,0.385668,0.000000,0.261877,46,577,623
301,302,2011-10-29,4,0,10,0,6,0,3,0.254167,0.227913,0.882500,0.351371,57,570,627
17,18,2011-01-18,1,0,1,0,2,1,2,0.216667,0.232333,0.861667,0.146775,9,674,683
340,341,2011-12-07,4,0,12,0,3,1,3,0.410000,0.400246,0.970417,0.266175,50,655,705
358,359,2011-12-25,1,0,12,0,0,0,1,0.274783,0.279961,0.681304,0.155091,303,451,754


In [22]:
dataset['cnt'][667] = dataset['cnt'][674]

C:\Users\Adam\Anaconda\envs\DeepLearningEnv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [25]:
dataset[667:668]

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
667,668,2012-10-29,4,1,10,0,1,1,3,0.44,0.4394,0.88,0.3582,2,20,5259


### One-hot encoding categorical variables

In [26]:
one_hot = pd.get_dummies(dataset['weekday'], prefix='weekday')
dataset = dataset.join(one_hot)

In [27]:
one_hot = pd.get_dummies(dataset['weathersit'], prefix='weathersit')
dataset = dataset.join(one_hot)

In [28]:
one_hot = pd.get_dummies(dataset['mnth'], prefix='mnth')
dataset = dataset.join(one_hot)

In [29]:
dataset

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,...,mnth_3,mnth_4,mnth_5,mnth_6,mnth_7,mnth_8,mnth_9,mnth_10,mnth_11,mnth_12
0,1,2011-01-01,1,0,1,0,6,0,2,0.344167,...,0,0,0,0,0,0,0,0,0,0
1,2,2011-01-02,1,0,1,0,0,0,2,0.363478,...,0,0,0,0,0,0,0,0,0,0
2,3,2011-01-03,1,0,1,0,1,1,1,0.196364,...,0,0,0,0,0,0,0,0,0,0
3,4,2011-01-04,1,0,1,0,2,1,1,0.200000,...,0,0,0,0,0,0,0,0,0,0
4,5,2011-01-05,1,0,1,0,3,1,1,0.226957,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,727,2012-12-27,1,1,12,0,4,1,2,0.254167,...,0,0,0,0,0,0,0,0,0,1
727,728,2012-12-28,1,1,12,0,5,1,2,0.253333,...,0,0,0,0,0,0,0,0,0,1
728,729,2012-12-29,1,1,12,0,6,0,2,0.253333,...,0,0,0,0,0,0,0,0,0,1
729,730,2012-12-30,1,1,12,0,0,0,1,0.255833,...,0,0,0,0,0,0,0,0,0,1


### Scaling

In [33]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(np.array(dataset['cnt']).reshape(len(dataset['cnt']), 1))
series = pd.DataFrame(scaled)
series.columns = ['cntscl']

In [35]:
dataset = pd.merge(dataset, series, left_index=True, right_index=True)

### Splitting data into train, validate, test

In [37]:
number_of_test_data = 50
number_of_holdout_data = 50
number_of_training_data = len(dataset) - number_of_holdout_data - number_of_test_data
print ("total, train, test, holdout:", len(dataset), number_of_training_data, number_of_test_data, number_of_holdout_data)

total, train, test, holdout: 731 631 50 50


In [38]:
datatrain = dataset[:number_of_training_data]
datatest = dataset[-(number_of_test_data+number_of_holdout_data):-number_of_holdout_data]
datahold = dataset[-number_of_holdout_data:]

### Preparing Training Data

In [52]:
in_seq1 = array(datatrain['holiday'])
in_seq2 = array(datatrain['workingday'])
in_seq3 = array(datatrain['temp'])
in_seq4 = array(datatrain['atemp'])
in_seq5 = array(datatrain['hum'])
in_seq6 = array(datatrain['windspeed'])
in_seq7 = array(datatrain['weekday_0'])
in_seq8 = array(datatrain['weekday_1'])
in_seq9 = array(datatrain['weekday_2'])
in_seq10 = array(datatrain['weekday_3'])
in_seq11 = array(datatrain['weekday_4'])
in_seq12 = array(datatrain['weekday_5'])
in_seq13 = array(datatrain['weekday_6'])
in_seq14 = array(datatrain['weathersit_1'])
in_seq15 = array(datatrain['weathersit_2'])
in_seq16 = array(datatrain['weathersit_3'])
out_seq_train = array(datatrain['cntscl'])

In [53]:
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
in_seq3 = in_seq3.reshape((len(in_seq3), 1))
in_seq4 = in_seq4.reshape((len(in_seq4), 1))
in_seq5 = in_seq5.reshape((len(in_seq5), 1))
in_seq6 = in_seq6.reshape((len(in_seq6), 1))
in_seq7 = in_seq7.reshape((len(in_seq7), 1))
in_seq8 = in_seq8.reshape((len(in_seq8), 1))
in_seq9 = in_seq9.reshape((len(in_seq9), 1))
in_seq10 = in_seq10.reshape((len(in_seq10), 1))
in_seq11 = in_seq11.reshape((len(in_seq11), 1))
in_seq12 = in_seq12.reshape((len(in_seq12), 1))
in_seq13 = in_seq13.reshape((len(in_seq13), 1))
in_seq14 = in_seq14.reshape((len(in_seq14), 1))
in_seq15 = in_seq15.reshape((len(in_seq15), 1))
in_seq16 = in_seq16.reshape((len(in_seq16), 1))
out_seq_train = out_seq_train.reshape((len(out_seq_train), 1))

In [54]:
datatrain_feed = hstack((in_seq1, in_seq2, in_seq3, in_seq4, in_seq5, in_seq6, in_seq7, in_seq8, in_seq9, in_seq10, in_seq11, in_seq12, in_seq13, in_seq14, in_seq15, in_seq16, out_seq_train))

### Preparing Validation Data

In [55]:
in_seq1 = array(datatest['holiday'])
in_seq2 = array(datatest['workingday'])
in_seq3 = array(datatest['temp'])
in_seq4 = array(datatest['atemp'])
in_seq5 = array(datatest['hum'])
in_seq6 = array(datatest['windspeed'])
in_seq7 = array(datatest['weekday_0'])
in_seq8 = array(datatest['weekday_1'])
in_seq9 = array(datatest['weekday_2'])
in_seq10 = array(datatest['weekday_3'])
in_seq11 = array(datatest['weekday_4'])
in_seq12 = array(datatest['weekday_5'])
in_seq13 = array(datatest['weekday_6'])
in_seq14 = array(datatest['weathersit_1'])
in_seq15 = array(datatest['weathersit_2'])
in_seq16 = array(datatest['weathersit_3'])
out_seq_test = array(datatest['cntscl'])

In [56]:
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
in_seq3 = in_seq3.reshape((len(in_seq3), 1))
in_seq4 = in_seq4.reshape((len(in_seq4), 1))
in_seq5 = in_seq5.reshape((len(in_seq5), 1))
in_seq6 = in_seq6.reshape((len(in_seq6), 1))
in_seq7 = in_seq7.reshape((len(in_seq7), 1))
in_seq8 = in_seq8.reshape((len(in_seq8), 1))
in_seq9 = in_seq9.reshape((len(in_seq9), 1))
in_seq10 = in_seq10.reshape((len(in_seq10), 1))
in_seq11 = in_seq11.reshape((len(in_seq11), 1))
in_seq12 = in_seq12.reshape((len(in_seq12), 1))
in_seq13 = in_seq13.reshape((len(in_seq13), 1))
in_seq14 = in_seq14.reshape((len(in_seq14), 1))
in_seq15 = in_seq15.reshape((len(in_seq15), 1))
in_seq16 = in_seq16.reshape((len(in_seq16), 1))
out_seq_test = out_seq_test.reshape((len(out_seq_test), 1))

In [57]:
datatest_feed = hstack((in_seq1, in_seq2, in_seq3, in_seq4, in_seq5, in_seq6, in_seq7, in_seq8, in_seq9, in_seq10, in_seq11, in_seq12, in_seq13, in_seq14, in_seq15, in_seq16, out_seq_test))

### Preparing Test Data

In [58]:
in_seq1 = array(datahold['holiday'])
in_seq2 = array(datahold['workingday'])
in_seq3 = array(datahold['temp'])
in_seq4 = array(datahold['atemp'])
in_seq5 = array(datahold['hum'])
in_seq6 = array(datahold['windspeed'])
in_seq7 = array(datahold['weekday_0'])
in_seq8 = array(datahold['weekday_1'])
in_seq9 = array(datahold['weekday_2'])
in_seq10 = array(datahold['weekday_3'])
in_seq11 = array(datahold['weekday_4'])
in_seq12 = array(datahold['weekday_5'])
in_seq13 = array(datahold['weekday_6'])
in_seq14 = array(datahold['weathersit_1'])
in_seq15 = array(datahold['weathersit_2'])
in_seq16 = array(datahold['weathersit_3'])
out_seq_hold = array(datahold['cntscl'])

In [59]:
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
in_seq3 = in_seq3.reshape((len(in_seq3), 1))
in_seq4 = in_seq4.reshape((len(in_seq4), 1))
in_seq5 = in_seq5.reshape((len(in_seq5), 1))
in_seq6 = in_seq6.reshape((len(in_seq6), 1))
in_seq7 = in_seq7.reshape((len(in_seq7), 1))
in_seq8 = in_seq8.reshape((len(in_seq8), 1))
in_seq9 = in_seq9.reshape((len(in_seq9), 1))
in_seq10 = in_seq10.reshape((len(in_seq10), 1))
in_seq11 = in_seq11.reshape((len(in_seq11), 1))
in_seq12 = in_seq12.reshape((len(in_seq12), 1))
in_seq13 = in_seq13.reshape((len(in_seq13), 1))
in_seq14 = in_seq14.reshape((len(in_seq14), 1))
in_seq15 = in_seq15.reshape((len(in_seq15), 1))
in_seq16 = in_seq16.reshape((len(in_seq16), 1))
out_seq_hold = out_seq_hold.reshape((len(out_seq_hold), 1))

In [60]:
datahold_feed = hstack((in_seq1, in_seq2, in_seq3, in_seq4, in_seq5, in_seq6, in_seq7, in_seq8, in_seq9, in_seq10, in_seq11, in_seq12, in_seq13, in_seq14, in_seq15, in_seq16, out_seq_hold))

In [65]:
n_features = datatrain_feed.shape[1]
n_input = 10
generator_train = TimeseriesGenerator(datatrain_feed, out_seq_train, length=n_input, batch_size=len(datatrain_feed))

In [66]:

for i in range(len(generator_train)):
	x, y = generator_train[i]
	print('%s => %s' % (x, y))

[[[0.         0.         0.344167   ... 1.         0.         0.06688398]
  [0.         0.         0.363478   ... 1.         0.         0.04466981]
  [0.         1.         0.196364   ... 0.         0.         0.11082941]
  ...
  [0.         0.         0.165      ... 1.         0.         0.06374502]
  [0.         0.         0.138333   ... 0.         0.         0.04720512]
  [0.         1.         0.150833   ... 0.         0.         0.10744899]]

 [[0.         0.         0.363478   ... 1.         0.         0.04466981]
  [0.         1.         0.196364   ... 0.         0.         0.11082941]
  [0.         1.         0.2        ... 0.         0.         0.13654473]
  ...
  [0.         0.         0.138333   ... 0.         0.         0.04720512]
  [0.         1.         0.150833   ... 0.         0.         0.10744899]
  [0.         1.         0.169091   ... 1.         0.         0.1004467 ]]

 [[0.         1.         0.196364   ... 0.         0.         0.11082941]
  [0.         1.      

In [67]:
generator_test = TimeseriesGenerator(datatest_feed, out_seq_test, length=n_input, batch_size=1)

In [68]:
for i in range(len(generator_test)):
	x, y = generator_test[i]
	print('%s => %s' % (x, y))

[[[0.         0.         0.529167   0.518933   0.467083   0.223258
   1.         0.         0.         0.         0.         0.
   0.         1.         0.         0.         0.90257153]
  [0.         1.         0.514167   0.502513   0.492917   0.142404
   0.         1.         0.         0.         0.         0.
   0.         1.         0.         0.         0.84570808]
  [0.         1.         0.55       0.544179   0.57       0.236321
   0.         0.         1.         0.         0.         0.
   0.         1.         0.         0.         0.85802246]
  [0.         1.         0.635      0.596613   0.630833   0.2444
   0.         0.         0.         1.         0.         0.
   0.         1.         0.         0.         0.88156465]
  [0.         1.         0.65       0.607975   0.690833   0.134342
   0.         0.         0.         0.         1.         0.
   0.         0.         1.         0.         0.84051672]
  [0.         1.         0.619167   0.585863   0.69       0.164179


In [69]:
generator_hold = TimeseriesGenerator(datahold_feed, out_seq_hold, length=n_input, batch_size=1)

In [70]:
for i in range(len(generator_hold)):
	x, y = generator_hold[i]
	print('%s => %s' % (x, y))

[[[1.         0.         0.485      0.475383   0.741667   0.173517
   0.         1.         0.         0.         0.         0.
   0.         1.         0.         0.         0.7048171 ]
  [0.         1.         0.343333   0.323225   0.662917   0.342046
   0.         0.         1.         0.         0.         0.
   0.         0.         1.         0.         0.44223108]
  [0.         1.         0.289167   0.281563   0.552083   0.199625
   0.         0.         0.         1.         0.         0.
   0.         1.         0.         0.         0.61137269]
  [0.         1.         0.321667   0.324492   0.620417   0.152987
   0.         0.         0.         0.         1.         0.
   0.         0.         1.         0.         0.60533623]
  [0.         1.         0.345      0.347204   0.524583   0.171025
   0.         0.         0.         0.         0.         1.
   0.         1.         0.         0.         0.63588072]
  [0.         0.         0.325      0.326383   0.545417   0.17972

### Creating the Simple RNN Model

In [71]:
print("timesteps, features:", n_input, n_features)

timesteps, features: 10 17


In [73]:
model = Sequential()

model.add(SimpleRNN(4, activation='relu', input_shape=(n_input, n_features), return_sequences = False))
model.add(Dense(1, activation='relu'))

adam = Adam(lr=0.0001)
model.compile(optimizer=adam, loss='mse')

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_2 (SimpleRNN)     (None, 4)                 88        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 5         
Total params: 93
Trainable params: 93
Non-trainable params: 0
_________________________________________________________________


### Training the Model

In [74]:
score = model.fit_generator(generator_train, epochs=3000, verbose=2, validation_data=generator_test)



Epoch 1/3000
 - 1s - loss: 0.2596 - val_loss: 0.4935
Epoch 2/3000
 - 0s - loss: 0.2593 - val_loss: 0.4925
Epoch 3/3000
 - 0s - loss: 0.2589 - val_loss: 0.4915
Epoch 4/3000
 - 0s - loss: 0.2585 - val_loss: 0.4905
Epoch 5/3000
 - 0s - loss: 0.2582 - val_loss: 0.4895
Epoch 6/3000
 - 0s - loss: 0.2578 - val_loss: 0.4886
Epoch 7/3000
 - 0s - loss: 0.2575 - val_loss: 0.4876
Epoch 8/3000
 - 0s - loss: 0.2571 - val_loss: 0.4866
Epoch 9/3000
 - 0s - loss: 0.2568 - val_loss: 0.4857
Epoch 10/3000
 - 0s - loss: 0.2564 - val_loss: 0.4847
Epoch 11/3000
 - 0s - loss: 0.2560 - val_loss: 0.4838
Epoch 12/3000
 - 0s - loss: 0.2557 - val_loss: 0.4829
Epoch 13/3000
 - 0s - loss: 0.2553 - val_loss: 0.4820
Epoch 14/3000
 - 0s - loss: 0.2550 - val_loss: 0.4810
Epoch 15/3000
 - 0s - loss: 0.2547 - val_loss: 0.4801
Epoch 16/3000
 - 0s - loss: 0.2543 - val_loss: 0.4792
Epoch 17/3000
 - 0s - loss: 0.2540 - val_loss: 0.4783
Epoch 18/3000
 - 0s - loss: 0.2536 - val_loss: 0.4774
Epoch 19/3000
 - 0s - loss: 0.2533 

Epoch 145/3000
 - 0s - loss: 0.2143 - val_loss: 0.3981
Epoch 146/3000
 - 0s - loss: 0.2141 - val_loss: 0.3975
Epoch 147/3000
 - 0s - loss: 0.2139 - val_loss: 0.3969
Epoch 148/3000
 - 0s - loss: 0.2136 - val_loss: 0.3963
Epoch 149/3000
 - 0s - loss: 0.2134 - val_loss: 0.3957
Epoch 150/3000
 - 0s - loss: 0.2131 - val_loss: 0.3951
Epoch 151/3000
 - 0s - loss: 0.2129 - val_loss: 0.3944
Epoch 152/3000
 - 0s - loss: 0.2126 - val_loss: 0.3938
Epoch 153/3000
 - 0s - loss: 0.2123 - val_loss: 0.3931
Epoch 154/3000
 - 0s - loss: 0.2121 - val_loss: 0.3925
Epoch 155/3000
 - 0s - loss: 0.2118 - val_loss: 0.3919
Epoch 156/3000
 - 0s - loss: 0.2116 - val_loss: 0.3912
Epoch 157/3000
 - 0s - loss: 0.2113 - val_loss: 0.3906
Epoch 158/3000
 - 0s - loss: 0.2110 - val_loss: 0.3900
Epoch 159/3000
 - 0s - loss: 0.2107 - val_loss: 0.3893
Epoch 160/3000
 - 0s - loss: 0.2104 - val_loss: 0.3887
Epoch 161/3000
 - 0s - loss: 0.2102 - val_loss: 0.3881
Epoch 162/3000
 - 0s - loss: 0.2099 - val_loss: 0.3874
Epoch 163/

Epoch 294/3000
 - 0s - loss: 0.1768 - val_loss: 0.3270
Epoch 295/3000
 - 0s - loss: 0.1766 - val_loss: 0.3267
Epoch 296/3000
 - 0s - loss: 0.1764 - val_loss: 0.3264
Epoch 297/3000
 - 0s - loss: 0.1762 - val_loss: 0.3261
Epoch 298/3000
 - 0s - loss: 0.1761 - val_loss: 0.3258
Epoch 299/3000
 - 0s - loss: 0.1759 - val_loss: 0.3255
Epoch 300/3000
 - 0s - loss: 0.1757 - val_loss: 0.3252
Epoch 301/3000
 - 0s - loss: 0.1755 - val_loss: 0.3249
Epoch 302/3000
 - 0s - loss: 0.1754 - val_loss: 0.3247
Epoch 303/3000
 - 0s - loss: 0.1752 - val_loss: 0.3244
Epoch 304/3000
 - 0s - loss: 0.1750 - val_loss: 0.3241
Epoch 305/3000
 - 0s - loss: 0.1749 - val_loss: 0.3238
Epoch 306/3000
 - 0s - loss: 0.1747 - val_loss: 0.3236
Epoch 307/3000
 - 0s - loss: 0.1745 - val_loss: 0.3233
Epoch 308/3000
 - 0s - loss: 0.1744 - val_loss: 0.3230
Epoch 309/3000
 - 0s - loss: 0.1742 - val_loss: 0.3228
Epoch 310/3000
 - 0s - loss: 0.1740 - val_loss: 0.3225
Epoch 311/3000
 - 0s - loss: 0.1739 - val_loss: 0.3223
Epoch 312/

Epoch 443/3000
 - 0s - loss: 0.1530 - val_loss: 0.2945
Epoch 444/3000
 - 0s - loss: 0.1529 - val_loss: 0.2943
Epoch 445/3000
 - 0s - loss: 0.1528 - val_loss: 0.2942
Epoch 446/3000
 - 0s - loss: 0.1526 - val_loss: 0.2941
Epoch 447/3000
 - 0s - loss: 0.1525 - val_loss: 0.2940
Epoch 448/3000
 - 0s - loss: 0.1524 - val_loss: 0.2939
Epoch 449/3000
 - 0s - loss: 0.1522 - val_loss: 0.2937
Epoch 450/3000
 - 0s - loss: 0.1521 - val_loss: 0.2936
Epoch 451/3000
 - 0s - loss: 0.1520 - val_loss: 0.2935
Epoch 452/3000
 - 0s - loss: 0.1519 - val_loss: 0.2934
Epoch 453/3000
 - 0s - loss: 0.1518 - val_loss: 0.2933
Epoch 454/3000
 - 0s - loss: 0.1517 - val_loss: 0.2932
Epoch 455/3000
 - 0s - loss: 0.1515 - val_loss: 0.2931
Epoch 456/3000
 - 0s - loss: 0.1514 - val_loss: 0.2930
Epoch 457/3000
 - 0s - loss: 0.1513 - val_loss: 0.2929
Epoch 458/3000
 - 0s - loss: 0.1512 - val_loss: 0.2928
Epoch 459/3000
 - 0s - loss: 0.1511 - val_loss: 0.2927
Epoch 460/3000
 - 0s - loss: 0.1510 - val_loss: 0.2926
Epoch 461/

Epoch 592/3000
 - 0s - loss: 0.1399 - val_loss: 0.2854
Epoch 593/3000
 - 0s - loss: 0.1398 - val_loss: 0.2854
Epoch 594/3000
 - 0s - loss: 0.1398 - val_loss: 0.2854
Epoch 595/3000
 - 0s - loss: 0.1397 - val_loss: 0.2853
Epoch 596/3000
 - 0s - loss: 0.1396 - val_loss: 0.2853
Epoch 597/3000
 - 0s - loss: 0.1395 - val_loss: 0.2853
Epoch 598/3000
 - 0s - loss: 0.1395 - val_loss: 0.2853
Epoch 599/3000
 - 0s - loss: 0.1394 - val_loss: 0.2853
Epoch 600/3000
 - 0s - loss: 0.1393 - val_loss: 0.2852
Epoch 601/3000
 - 0s - loss: 0.1393 - val_loss: 0.2852
Epoch 602/3000
 - 0s - loss: 0.1392 - val_loss: 0.2852
Epoch 603/3000
 - 0s - loss: 0.1391 - val_loss: 0.2852
Epoch 604/3000
 - 0s - loss: 0.1391 - val_loss: 0.2852
Epoch 605/3000
 - 0s - loss: 0.1390 - val_loss: 0.2852
Epoch 606/3000
 - 0s - loss: 0.1389 - val_loss: 0.2851
Epoch 607/3000
 - 0s - loss: 0.1389 - val_loss: 0.2851
Epoch 608/3000
 - 0s - loss: 0.1388 - val_loss: 0.2851
Epoch 609/3000
 - 0s - loss: 0.1387 - val_loss: 0.2850
Epoch 610/

...

Epoch 2361/3000
 - 0s - loss: 0.0710 - val_loss: 0.1235
Epoch 2362/3000
 - 0s - loss: 0.0710 - val_loss: 0.1234
Epoch 2363/3000
 - 0s - loss: 0.0710 - val_loss: 0.1234
Epoch 2364/3000
 - 0s - loss: 0.0709 - val_loss: 0.1233
Epoch 2365/3000
 - 0s - loss: 0.0709 - val_loss: 0.1233
Epoch 2366/3000
 - 0s - loss: 0.0709 - val_loss: 0.1232
Epoch 2367/3000
 - 0s - loss: 0.0708 - val_loss: 0.1232
Epoch 2368/3000
 - 0s - loss: 0.0708 - val_loss: 0.1231
Epoch 2369/3000
 - 0s - loss: 0.0708 - val_loss: 0.1231
Epoch 2370/3000
 - 0s - loss: 0.0707 - val_loss: 0.1230
Epoch 2371/3000
 - 0s - loss: 0.0707 - val_loss: 0.1230
Epoch 2372/3000
 - 0s - loss: 0.0707 - val_loss: 0.1229
Epoch 2373/3000
 - 0s - loss: 0.0707 - val_loss: 0.1229
Epoch 2374/3000
 - 0s - loss: 0.0706 - val_loss: 0.1228
Epoch 2375/3000
 - 0s - loss: 0.0706 - val_loss: 0.1228
Epoch 2376/3000
 - 0s - loss: 0.0706 - val_loss: 0.1227
Epoch 2377/3000
 - 0s - loss: 0.0705 - val_loss: 0.1227
Epoch 2378/3000
 - 0s - loss: 0.0705 - val_loss:

Epoch 2508/3000
 - 0s - loss: 0.0673 - val_loss: 0.1172
Epoch 2509/3000
 - 0s - loss: 0.0673 - val_loss: 0.1172
Epoch 2510/3000
 - 0s - loss: 0.0673 - val_loss: 0.1171
Epoch 2511/3000
 - 0s - loss: 0.0672 - val_loss: 0.1171
Epoch 2512/3000
 - 0s - loss: 0.0672 - val_loss: 0.1171
Epoch 2513/3000
 - 0s - loss: 0.0672 - val_loss: 0.1170
Epoch 2514/3000
 - 0s - loss: 0.0672 - val_loss: 0.1170
Epoch 2515/3000
 - 0s - loss: 0.0672 - val_loss: 0.1170
Epoch 2516/3000
 - 0s - loss: 0.0671 - val_loss: 0.1170
Epoch 2517/3000
 - 0s - loss: 0.0671 - val_loss: 0.1169
Epoch 2518/3000
 - 0s - loss: 0.0671 - val_loss: 0.1169
Epoch 2519/3000
 - 0s - loss: 0.0671 - val_loss: 0.1169
Epoch 2520/3000
 - 0s - loss: 0.0671 - val_loss: 0.1168
Epoch 2521/3000
 - 0s - loss: 0.0670 - val_loss: 0.1168
Epoch 2522/3000
 - 0s - loss: 0.0670 - val_loss: 0.1168
Epoch 2523/3000
 - 0s - loss: 0.0670 - val_loss: 0.1167
Epoch 2524/3000
 - 0s - loss: 0.0670 - val_loss: 0.1167
Epoch 2525/3000
 - 0s - loss: 0.0669 - val_loss:

Epoch 2655/3000
 - 0s - loss: 0.0633 - val_loss: 0.1111
Epoch 2656/3000
 - 0s - loss: 0.0633 - val_loss: 0.1110
Epoch 2657/3000
 - 0s - loss: 0.0632 - val_loss: 0.1109
Epoch 2658/3000
 - 0s - loss: 0.0632 - val_loss: 0.1108
Epoch 2659/3000
 - 0s - loss: 0.0631 - val_loss: 0.1107
Epoch 2660/3000
 - 0s - loss: 0.0631 - val_loss: 0.1106
Epoch 2661/3000
 - 0s - loss: 0.0630 - val_loss: 0.1105
Epoch 2662/3000
 - 0s - loss: 0.0630 - val_loss: 0.1104
Epoch 2663/3000
 - 0s - loss: 0.0629 - val_loss: 0.1103
Epoch 2664/3000
 - 0s - loss: 0.0629 - val_loss: 0.1103
Epoch 2665/3000
 - 0s - loss: 0.0628 - val_loss: 0.1102
Epoch 2666/3000
 - 0s - loss: 0.0628 - val_loss: 0.1101
Epoch 2667/3000
 - 0s - loss: 0.0627 - val_loss: 0.1100
Epoch 2668/3000
 - 0s - loss: 0.0627 - val_loss: 0.1099
Epoch 2669/3000
 - 0s - loss: 0.0626 - val_loss: 0.1098
Epoch 2670/3000
 - 0s - loss: 0.0626 - val_loss: 0.1097
Epoch 2671/3000
 - 0s - loss: 0.0626 - val_loss: 0.1096
Epoch 2672/3000
 - 0s - loss: 0.0625 - val_loss:

Epoch 2802/3000
 - 0s - loss: 0.0522 - val_loss: 0.0879
Epoch 2803/3000
 - 0s - loss: 0.0521 - val_loss: 0.0877
Epoch 2804/3000
 - 0s - loss: 0.0521 - val_loss: 0.0876
Epoch 2805/3000
 - 0s - loss: 0.0520 - val_loss: 0.0874
Epoch 2806/3000
 - 0s - loss: 0.0519 - val_loss: 0.0872
Epoch 2807/3000
 - 0s - loss: 0.0518 - val_loss: 0.0871
Epoch 2808/3000
 - 0s - loss: 0.0517 - val_loss: 0.0869
Epoch 2809/3000
 - 0s - loss: 0.0516 - val_loss: 0.0868
Epoch 2810/3000
 - 0s - loss: 0.0515 - val_loss: 0.0866
Epoch 2811/3000
 - 0s - loss: 0.0514 - val_loss: 0.0864
Epoch 2812/3000
 - 0s - loss: 0.0513 - val_loss: 0.0862
Epoch 2813/3000
 - 0s - loss: 0.0512 - val_loss: 0.0861
Epoch 2814/3000
 - 0s - loss: 0.0511 - val_loss: 0.0859
Epoch 2815/3000
 - 0s - loss: 0.0510 - val_loss: 0.0857
Epoch 2816/3000
 - 0s - loss: 0.0509 - val_loss: 0.0855
Epoch 2817/3000
 - 0s - loss: 0.0508 - val_loss: 0.0853
Epoch 2818/3000
 - 0s - loss: 0.0507 - val_loss: 0.0852
Epoch 2819/3000
 - 0s - loss: 0.0506 - val_loss:

Epoch 2949/3000
 - 0s - loss: 0.0398 - val_loss: 0.0678
Epoch 2950/3000
 - 0s - loss: 0.0398 - val_loss: 0.0677
Epoch 2951/3000
 - 0s - loss: 0.0397 - val_loss: 0.0676
Epoch 2952/3000
 - 0s - loss: 0.0396 - val_loss: 0.0675
Epoch 2953/3000
 - 0s - loss: 0.0396 - val_loss: 0.0674
Epoch 2954/3000
 - 0s - loss: 0.0395 - val_loss: 0.0673
Epoch 2955/3000
 - 0s - loss: 0.0394 - val_loss: 0.0672
Epoch 2956/3000
 - 0s - loss: 0.0394 - val_loss: 0.0671
Epoch 2957/3000
 - 0s - loss: 0.0393 - val_loss: 0.0671
Epoch 2958/3000
 - 0s - loss: 0.0392 - val_loss: 0.0670
Epoch 2959/3000
 - 0s - loss: 0.0392 - val_loss: 0.0669
Epoch 2960/3000
 - 0s - loss: 0.0391 - val_loss: 0.0668
Epoch 2961/3000
 - 0s - loss: 0.0391 - val_loss: 0.0667
Epoch 2962/3000
 - 0s - loss: 0.0390 - val_loss: 0.0666
Epoch 2963/3000
 - 0s - loss: 0.0389 - val_loss: 0.0665
Epoch 2964/3000
 - 0s - loss: 0.0389 - val_loss: 0.0665
Epoch 2965/3000
 - 0s - loss: 0.0388 - val_loss: 0.0664
Epoch 2966/3000
 - 0s - loss: 0.0388 - val_loss:

### Training and Validation Loss Functions

In [102]:
losses = score.history['loss']
val_losses = score.history['val_loss']
plt.figure(figsize=(10,5))
plt.plot(losses, label="trainset", color = "gray")
plt.plot(val_losses, label="validationset", color="orange")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()

C:\Users\Adam\Anaconda\envs\DeepLearningEnv\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  This is separate from the ipykernel package so we can avoid doing imports until


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Validation Predictions and Errors

In [76]:
df_result = pd.DataFrame({'Actual' : [], 'Prediction' : []})

for i in range(len(generator_test)):
    x, y = generator_test[i]
    x_input = array(x).reshape((1, n_input, n_features))
    yhat = model.predict(x_input, verbose=2)
    df_result = df_result.append({'Actual': scaler.inverse_transform(y)[0][0], 'Prediction': scaler.inverse_transform(yhat)[0][0]}, ignore_index=True)

In [88]:
df_result['Error'] = 100 * (df_result['Prediction'] - df_result['Actual']) / df_result['Actual']

In [93]:
plt.figure(figsize=(8,5))
plt.ylim(0,10000)
plt.plot(df_result['Actual'], color='black')
plt.plot(df_result['Prediction'], color='skyblue')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [99]:
plt.figure(figsize=(8,5))
plt.ylim(-600,600)
plt.plot(df_result['Error'], color='orange')
plt.show()

C:\Users\Adam\Anaconda\envs\DeepLearningEnv\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Test Predictions and Errors

In [103]:
df_result = pd.DataFrame({'Actual' : [], 'Prediction' : []})

for i in range(len(generator_hold)):
    x, y = generator_hold[i]
    x_input = array(x).reshape((1, n_input, n_features))
    yhat = model.predict(x_input, verbose=2)
    df_result = df_result.append({'Actual': scaler.inverse_transform(y)[0][0], 'Prediction': scaler.inverse_transform(yhat)[0][0]}, ignore_index=True)

In [104]:
df_result['Error'] = 100 * (df_result['Prediction'] - df_result['Actual']) / df_result['Actual']

In [105]:
plt.figure(figsize=(8,5))
plt.ylim(0,10000)
plt.plot(df_result['Actual'], color='black')
plt.plot(df_result['Prediction'], color='skyblue')
plt.show()

C:\Users\Adam\Anaconda\envs\DeepLearningEnv\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [106]:
plt.figure(figsize=(8,5))
plt.ylim(-600,600)
plt.plot(df_result['Error'], color='orange')
plt.show()

C:\Users\Adam\Anaconda\envs\DeepLearningEnv\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Try with months

In [107]:
### Preparing Training Data

in_seq1 = array(datatrain['holiday'])
in_seq2 = array(datatrain['workingday'])
in_seq3 = array(datatrain['temp'])
in_seq4 = array(datatrain['atemp'])
in_seq5 = array(datatrain['hum'])
in_seq6 = array(datatrain['windspeed'])
in_seq7 = array(datatrain['weekday_0'])
in_seq8 = array(datatrain['weekday_1'])
in_seq9 = array(datatrain['weekday_2'])
in_seq10 = array(datatrain['weekday_3'])
in_seq11 = array(datatrain['weekday_4'])
in_seq12 = array(datatrain['weekday_5'])
in_seq13 = array(datatrain['weekday_6'])
in_seq14 = array(datatrain['weathersit_1'])
in_seq15 = array(datatrain['weathersit_2'])
in_seq16 = array(datatrain['weathersit_3'])
in_seq17 = array(datatrain['mnth_1'])
in_seq18 = array(datatrain['mnth_2'])
in_seq19 = array(datatrain['mnth_3'])
in_seq20 = array(datatrain['mnth_4'])
in_seq21 = array(datatrain['mnth_5'])
in_seq22 = array(datatrain['mnth_6'])
in_seq23 = array(datatrain['mnth_7'])
in_seq24 = array(datatrain['mnth_8'])
in_seq25 = array(datatrain['mnth_9'])
in_seq26 = array(datatrain['mnth_10'])
in_seq27 = array(datatrain['mnth_11'])
in_seq28 = array(datatrain['mnth_12'])
out_seq_train = array(datatrain['cntscl'])

in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
in_seq3 = in_seq3.reshape((len(in_seq3), 1))
in_seq4 = in_seq4.reshape((len(in_seq4), 1))
in_seq5 = in_seq5.reshape((len(in_seq5), 1))
in_seq6 = in_seq6.reshape((len(in_seq6), 1))
in_seq7 = in_seq7.reshape((len(in_seq7), 1))
in_seq8 = in_seq8.reshape((len(in_seq8), 1))
in_seq9 = in_seq9.reshape((len(in_seq9), 1))
in_seq10 = in_seq10.reshape((len(in_seq10), 1))
in_seq11 = in_seq11.reshape((len(in_seq11), 1))
in_seq12 = in_seq12.reshape((len(in_seq12), 1))
in_seq13 = in_seq13.reshape((len(in_seq13), 1))
in_seq14 = in_seq14.reshape((len(in_seq14), 1))
in_seq15 = in_seq15.reshape((len(in_seq15), 1))
in_seq16 = in_seq16.reshape((len(in_seq16), 1))
in_seq17 = in_seq17.reshape((len(in_seq17), 1))
in_seq18 = in_seq18.reshape((len(in_seq18), 1))
in_seq19 = in_seq19.reshape((len(in_seq19), 1))
in_seq20 = in_seq20.reshape((len(in_seq20), 1))
in_seq21 = in_seq21.reshape((len(in_seq21), 1))
in_seq22 = in_seq22.reshape((len(in_seq22), 1))
in_seq23 = in_seq23.reshape((len(in_seq23), 1))
in_seq24 = in_seq24.reshape((len(in_seq24), 1))
in_seq25 = in_seq25.reshape((len(in_seq25), 1))
in_seq26 = in_seq26.reshape((len(in_seq26), 1))
in_seq27 = in_seq27.reshape((len(in_seq27), 1))
in_seq28 = in_seq28.reshape((len(in_seq28), 1))
out_seq_train = out_seq_train.reshape((len(out_seq_train), 1))

datatrain_feedmo = hstack((in_seq1, in_seq2, in_seq3, in_seq4, in_seq5, in_seq6, in_seq7, in_seq8, in_seq9, in_seq10, 
                           in_seq11, in_seq12, in_seq13, in_seq14, in_seq15, in_seq16, in_seq17, in_seq18, in_seq19, in_seq20, 
                           in_seq21, in_seq22, in_seq23, in_seq24, in_seq25, in_seq26, in_seq27, in_seq28, out_seq_train))

### Preparing Validation Data

in_seq1 = array(datatest['holiday'])
in_seq2 = array(datatest['workingday'])
in_seq3 = array(datatest['temp'])
in_seq4 = array(datatest['atemp'])
in_seq5 = array(datatest['hum'])
in_seq6 = array(datatest['windspeed'])
in_seq7 = array(datatest['weekday_0'])
in_seq8 = array(datatest['weekday_1'])
in_seq9 = array(datatest['weekday_2'])
in_seq10 = array(datatest['weekday_3'])
in_seq11 = array(datatest['weekday_4'])
in_seq12 = array(datatest['weekday_5'])
in_seq13 = array(datatest['weekday_6'])
in_seq14 = array(datatest['weathersit_1'])
in_seq15 = array(datatest['weathersit_2'])
in_seq16 = array(datatest['weathersit_3'])
in_seq17 = array(datatest['mnth_1'])
in_seq18 = array(datatest['mnth_2'])
in_seq19 = array(datatest['mnth_3'])
in_seq20 = array(datatest['mnth_4'])
in_seq21 = array(datatest['mnth_5'])
in_seq22 = array(datatest['mnth_6'])
in_seq23 = array(datatest['mnth_7'])
in_seq24 = array(datatest['mnth_8'])
in_seq25 = array(datatest['mnth_9'])
in_seq26 = array(datatest['mnth_10'])
in_seq27 = array(datatest['mnth_11'])
in_seq28 = array(datatest['mnth_12'])
out_seq_test = array(datatest['cntscl'])

in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
in_seq3 = in_seq3.reshape((len(in_seq3), 1))
in_seq4 = in_seq4.reshape((len(in_seq4), 1))
in_seq5 = in_seq5.reshape((len(in_seq5), 1))
in_seq6 = in_seq6.reshape((len(in_seq6), 1))
in_seq7 = in_seq7.reshape((len(in_seq7), 1))
in_seq8 = in_seq8.reshape((len(in_seq8), 1))
in_seq9 = in_seq9.reshape((len(in_seq9), 1))
in_seq10 = in_seq10.reshape((len(in_seq10), 1))
in_seq11 = in_seq11.reshape((len(in_seq11), 1))
in_seq12 = in_seq12.reshape((len(in_seq12), 1))
in_seq13 = in_seq13.reshape((len(in_seq13), 1))
in_seq14 = in_seq14.reshape((len(in_seq14), 1))
in_seq15 = in_seq15.reshape((len(in_seq15), 1))
in_seq16 = in_seq16.reshape((len(in_seq16), 1))
in_seq17 = in_seq17.reshape((len(in_seq17), 1))
in_seq18 = in_seq18.reshape((len(in_seq18), 1))
in_seq19 = in_seq19.reshape((len(in_seq19), 1))
in_seq20 = in_seq20.reshape((len(in_seq20), 1))
in_seq21 = in_seq21.reshape((len(in_seq21), 1))
in_seq22 = in_seq22.reshape((len(in_seq22), 1))
in_seq23 = in_seq23.reshape((len(in_seq23), 1))
in_seq24 = in_seq24.reshape((len(in_seq24), 1))
in_seq25 = in_seq25.reshape((len(in_seq25), 1))
in_seq26 = in_seq26.reshape((len(in_seq26), 1))
in_seq27 = in_seq27.reshape((len(in_seq27), 1))
in_seq28 = in_seq28.reshape((len(in_seq28), 1))
out_seq_test = out_seq_test.reshape((len(out_seq_train), 1))

datatest_feedmo = hstack((in_seq1, in_seq2, in_seq3, in_seq4, in_seq5, in_seq6, in_seq7, in_seq8, in_seq9, in_seq10, 
                           in_seq11, in_seq12, in_seq13, in_seq14, in_seq15, in_seq16, in_seq17, in_seq18, in_seq19, in_seq20, 
                           in_seq21, in_seq22, in_seq23, in_seq24, in_seq25, in_seq26, in_seq27, in_seq28, out_seq_test))
### Preparing Test Data

in_seq1 = array(datahold['holiday'])
in_seq2 = array(datahold['workingday'])
in_seq3 = array(datahold['temp'])
in_seq4 = array(datahold['atemp'])
in_seq5 = array(datahold['hum'])
in_seq6 = array(datahold['windspeed'])
in_seq7 = array(datahold['weekday_0'])
in_seq8 = array(datahold['weekday_1'])
in_seq9 = array(datahold['weekday_2'])
in_seq10 = array(datahold['weekday_3'])
in_seq11 = array(datahold['weekday_4'])
in_seq12 = array(datahold['weekday_5'])
in_seq13 = array(datahold['weekday_6'])
in_seq14 = array(datahold['weathersit_1'])
in_seq15 = array(datahold['weathersit_2'])
in_seq16 = array(datahold['weathersit_3'])
in_seq17 = array(datahold['mnth_1'])
in_seq18 = array(datahold['mnth_2'])
in_seq19 = array(datahold['mnth_3'])
in_seq20 = array(datahold['mnth_4'])
in_seq21 = array(datahold['mnth_5'])
in_seq22 = array(datahold['mnth_6'])
in_seq23 = array(datahold['mnth_7'])
in_seq24 = array(datahold['mnth_8'])
in_seq25 = array(datahold['mnth_9'])
in_seq26 = array(datahold['mnth_10'])
in_seq27 = array(datahold['mnth_11'])
in_seq28 = array(datahold['mnth_12'])
out_seq_test = array(datahold['cntscl'])

in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
in_seq3 = in_seq3.reshape((len(in_seq3), 1))
in_seq4 = in_seq4.reshape((len(in_seq4), 1))
in_seq5 = in_seq5.reshape((len(in_seq5), 1))
in_seq6 = in_seq6.reshape((len(in_seq6), 1))
in_seq7 = in_seq7.reshape((len(in_seq7), 1))
in_seq8 = in_seq8.reshape((len(in_seq8), 1))
in_seq9 = in_seq9.reshape((len(in_seq9), 1))
in_seq10 = in_seq10.reshape((len(in_seq10), 1))
in_seq11 = in_seq11.reshape((len(in_seq11), 1))
in_seq12 = in_seq12.reshape((len(in_seq12), 1))
in_seq13 = in_seq13.reshape((len(in_seq13), 1))
in_seq14 = in_seq14.reshape((len(in_seq14), 1))
in_seq15 = in_seq15.reshape((len(in_seq15), 1))
in_seq16 = in_seq16.reshape((len(in_seq16), 1))
in_seq17 = in_seq17.reshape((len(in_seq17), 1))
in_seq18 = in_seq18.reshape((len(in_seq18), 1))
in_seq19 = in_seq19.reshape((len(in_seq19), 1))
in_seq20 = in_seq20.reshape((len(in_seq20), 1))
in_seq21 = in_seq21.reshape((len(in_seq21), 1))
in_seq22 = in_seq22.reshape((len(in_seq22), 1))
in_seq23 = in_seq23.reshape((len(in_seq23), 1))
in_seq24 = in_seq24.reshape((len(in_seq24), 1))
in_seq25 = in_seq25.reshape((len(in_seq25), 1))
in_seq26 = in_seq26.reshape((len(in_seq26), 1))
in_seq27 = in_seq27.reshape((len(in_seq27), 1))
in_seq28 = in_seq28.reshape((len(in_seq28), 1))
out_seq_hold = out_seq_hold.reshape((len(out_seq_train), 1))

datahold_feedmo = hstack((in_seq1, in_seq2, in_seq3, in_seq4, in_seq5, in_seq6, in_seq7, in_seq8, in_seq9, in_seq10, 
                           in_seq11, in_seq12, in_seq13, in_seq14, in_seq15, in_seq16, in_seq17, in_seq18, in_seq19, in_seq20, 
                           in_seq21, in_seq22, in_seq23, in_seq24, in_seq25, in_seq26, in_seq27, in_seq28, out_seq_hold))




ValueError: cannot reshape array of size 50 into shape (631,1)

In [ ]:
n_features = datatrain_feed.shape[1]
n_input = 10
generator_train = TimeseriesGenerator(datatrain_feed, out_seq_train, length=n_input, batch_size=len(datatrain_feed))


for i in range(len(generator_train)):
	x, y = generator_train[i]
	print('%s => %s' % (x, y))

generator_test = TimeseriesGenerator(datatest_feed, out_seq_test, length=n_input, batch_size=1)

for i in range(len(generator_test)):
	x, y = generator_test[i]
	print('%s => %s' % (x, y))

generator_hold = TimeseriesGenerator(datahold_feed, out_seq_hold, length=n_input, batch_size=1)

for i in range(len(generator_hold)):
	x, y = generator_hold[i]
	print('%s => %s' % (x, y))

### Creating the Simple RNN Model

print("timesteps, features:", n_input, n_features)

model = Sequential()

model.add(SimpleRNN(4, activation='relu', input_shape=(n_input, n_features), return_sequences = False))
model.add(Dense(1, activation='relu'))

adam = Adam(lr=0.0001)
model.compile(optimizer=adam, loss='mse')

model.summary()

### Training the Model

score = model.fit_generator(generator_train, epochs=3000, verbose=2, validation_data=generator_test)

### Training and Validation Loss Functions

losses = score.history['loss']
val_losses = score.history['val_loss']
plt.figure(figsize=(10,5))
plt.plot(losses, label="trainset", color = "gray")
plt.plot(val_losses, label="validationset", color="orange")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()

### Validation Predictions and Errors

df_result = pd.DataFrame({'Actual' : [], 'Prediction' : []})

for i in range(len(generator_test)):
    x, y = generator_test[i]
    x_input = array(x).reshape((1, n_input, n_features))
    yhat = model.predict(x_input, verbose=2)
    df_result = df_result.append({'Actual': scaler.inverse_transform(y)[0][0], 'Prediction': scaler.inverse_transform(yhat)[0][0]}, ignore_index=True)

df_result['Error'] = 100 * (df_result['Prediction'] - df_result['Actual']) / df_result['Actual']

plt.figure(figsize=(8,5))
plt.ylim(0,10000)
plt.plot(df_result['Actual'], color='black')
plt.plot(df_result['Prediction'], color='skyblue')
plt.show()

plt.figure(figsize=(8,5))
plt.ylim(-600,600)
plt.plot(df_result['Error'], color='orange')
plt.show()

### Test Predictions and Errors

df_result = pd.DataFrame({'Actual' : [], 'Prediction' : []})

for i in range(len(generator_hold)):
    x, y = generator_hold[i]
    x_input = array(x).reshape((1, n_input, n_features))
    yhat = model.predict(x_input, verbose=2)
    df_result = df_result.append({'Actual': scaler.inverse_transform(y)[0][0], 'Prediction': scaler.inverse_transform(yhat)[0][0]}, ignore_index=True)

df_result['Error'] = 100 * (df_result['Prediction'] - df_result['Actual']) / df_result['Actual']

plt.figure(figsize=(8,5))
plt.ylim(0,10000)
plt.plot(df_result['Actual'], color='black')
plt.plot(df_result['Prediction'], color='skyblue')
plt.show()

plt.figure(figsize=(8,5))
plt.ylim(-600,600)
plt.plot(df_result['Error'], color='orange')
plt.show()